In [ ]:
!pip install rdkit

In [ ]:
!pip install tensorflow

In [4]:
import numpy as np
import pandas as pd
from collections import OrderedDict
from itertools import chain
from rdkit import Chem
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM

2024-03-26 20:37:27.774871: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 20:37:27.775031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 20:37:27.955527: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
df = pd.read_csv('/kaggle/input/1sfgerterg/.csv')

In [18]:
train_data = np.array(df['smiles'])

In [20]:
unique_chars = sorted(list(OrderedDict.fromkeys(chain.from_iterable(train_data))))
char_to_int = dict((c, i) for i, c in enumerate(unique_chars))

In [21]:
int_to_char = dict((i, c) for i, c in enumerate(unique_chars))

In [22]:
char_to_int.update({"E" : len(char_to_int)})
int_to_char.update({len(int_to_char) : "E"})

In [23]:
mapping_size = len(char_to_int)
reverse_mapping_size = len(int_to_char)
print ("Size of the character to integer dictionary is: ", mapping_size)
print ("Size of the integer to character dictionary is: ", reverse_mapping_size)

Size of the character to integer dictionary is:  32
Size of the integer to character dictionary is:  32


In [24]:
def gen_data(data, int_to_char, char_to_int, embed):
    one_hot =  np.zeros((data.shape[0], embed+1, len(char_to_int)),dtype=np.int8)
    for i,smile in enumerate(data):
        for j,c in enumerate(smile):
            one_hot[i,j,char_to_int[c]] = 1
        one_hot[i,len(smile):,char_to_int["E"]] = 1
    return one_hot[:,0:-1,:], one_hot[:,1:,:]

In [25]:
embed = max([len(seq) for seq in train_data])

X, Y = gen_data(train_data, int_to_char, char_to_int, embed)
X, Y = shuffle(X, Y)

In [29]:
model = Sequential()
model.add(LSTM(256, input_shape=(None, X.shape[2]), return_sequences = True))
model.add(Dropout(0.25))
model.add(LSTM(256, return_sequences = True))
model.add(Dropout(0.25))
model.add(Dense(Y.shape[-1], activation='softmax'))
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, None, 256)         295936    
                                                                 
 dropout_2 (Dropout)         (None, None, 256)         0         
                                                                 
 lstm_3 (LSTM)               (None, None, 256)         525312    
                                                                 
 dropout_3 (Dropout)         (None, None, 256)         0         
                                                                 
 dense_1 (Dense)             (None, None, 32)          8224      
                                                                 
Total params: 829472 (3.16 MB)
Trainable params: 829472 (3.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [30]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam')
history = model.fit(X, Y, epochs = 20, batch_size = 60)

Epoch 1/20
5/5 [==============================] - 8s 569ms/step - loss: 2.8845
Epoch 2/20
5/5 [==============================] - 3s 558ms/step - loss: 1.3668
Epoch 3/20
5/5 [==============================] - 3s 569ms/step - loss: 1.2501
Epoch 4/20
5/5 [==============================] - 3s 561ms/step - loss: 1.1071
Epoch 5/20
5/5 [==============================] - 3s 566ms/step - loss: 1.0229
Epoch 6/20
5/5 [==============================] - 3s 577ms/step - loss: 0.9514
Epoch 7/20
5/5 [==============================] - 3s 561ms/step - loss: 0.9037
Epoch 8/20
5/5 [==============================] - 3s 550ms/step - loss: 0.8781
Epoch 9/20
5/5 [==============================] - 3s 545ms/step - loss: 0.8512
Epoch 10/20
5/5 [==============================] - 3s 567ms/step - loss: 0.8420
Epoch 11/20
5/5 [==============================] - 3s 624ms/step - loss: 0.8273
Epoch 12/20
5/5 [==============================] - 3s 557ms/step - loss: 0.8165
Epoch 13/20
5/5 [==============================] 

In [33]:
def gen_mol(model, start_char):
    preds = []
    
    x = np.zeros((1,1,32))
    x[0,0,char_to_int[start_char]] = 1
    stringc = start_char
    
    pred = model.predict(x).flatten()
    ch = int_to_char[np.random.choice(np.arange(0,len(char_to_int)), p=pred)]
    preds.append(pred)
    stringc += ch
    x1 = np.zeros((1,1,32))
    x1[0,0,char_to_int[ch]] = 1
    x = np.append(x,x1,axis=1)
    
    while ch !='E':
        pred = model.predict(np.expand_dims(x[:,-1,:],axis = 1)).flatten()
        ch = int_to_char[np.random.choice(np.arange(0,len(char_to_int)), size=None, replace=True, p=pred)]
        preds.append(pred)

        x1 = np.zeros((1,1,32))
        x1[0,0,char_to_int[ch]] = 1
        x = np.append(x,x1,axis=1)
        stringc += ch
    
    print(stringc)
    return np.array(preds)

In [37]:
df.sort_values(by='lgK')

,id,smiles,lgK
234,236,O=CO,0.99
238,240,O=C(O)CCl,1.02
239,241,O=C(O)CI,1.32
42,42,O=[N+]([O-])c1ccc(O)cc1,1.46
242,244,O=C(O)c1cccc([N+](=O)[O-])c1,1.64
...,...,...,...
71,72,O=C(O)CN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(CC(=O)O)C...,24.26
136,137,O=P(O)(O)CN1CCN(CP(=O)(O)O)CCN(CP(=O)(O)O)CCN(...,24.78
150,152,O=C(O)CCP(=O)(O)CN1CCN(C(=O)O)CCN(C(=O)O)CCN(C...,25.50
149,151,O=C(O)N1CCN(CP(=O)(O)O)CCN(C(=O)O)CCN(C(=O)O)CC1,28.60


In [40]:
preds = gen_mol(model, 'C')

1/1 [==============================] - 0s 25ms/step
C5N-Po32@r1O1E
